In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle\.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [01:17<00:00, 45.4MB/s]
100% 2.70G/2.70G [01:17<00:00, 37.5MB/s]


In [6]:
!unzip -q new-plant-diseases-dataset.zip

# Import Libraries

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

# Define Batch Specifications

In [8]:
## Defining batch specfications

# batch_size = 100
batch_size = 32 
img_height = 250
img_width = 250

In [9]:
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Load the Dataset

In [10]:
training_data = tf.keras.preprocessing.image_dataset_from_directory("/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 70295 files belonging to 38 classes.


In [12]:
new_training_data = tf.keras.preprocessing.image_dataset_from_directory("/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 70295 files belonging to 38 classes.


# Getting the Validation Data

In [13]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory("/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 17572 files belonging to 38 classes.


# Storing the Label Names

In [14]:
target_names = training_data.class_names
print(target_names)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

# Building ResNet50 Architecture

In [15]:
# Building ResNet-50 Architecture
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Dense(len(target_names), activation='softmax'))
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 4096)              536875008 
                                                                 
 batch_normalization (BatchN  (None, 4096)             16384     
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 batch_normalization_1 (Batc  (None, 4096)             16

# Compile the Model

In [16]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training and Validation

In [17]:
history = model.fit(training_data,validation_data= validation_data,epochs = 10)

Epoch 1/10
2197/2197 [==============================] - 868s 379ms/step - loss: 1.5008 - accuracy: 0.7262 - val_loss: 3.1056 - val_accuracy: 0.5499
Epoch 2/10
2197/2197 [==============================] - 811s 369ms/step - loss: 0.4390 - accuracy: 0.8960 - val_loss: 1.5400 - val_accuracy: 0.7322
Epoch 3/10
2197/2197 [==============================] - 808s 368ms/step - loss: 0.3131 - accuracy: 0.9263 - val_loss: 0.6311 - val_accuracy: 0.8809
Epoch 4/10
2197/2197 [==============================] - 808s 367ms/step - loss: 0.1862 - accuracy: 0.9507 - val_loss: 0.6116 - val_accuracy: 0.8747
Epoch 5/10
2197/2197 [==============================] - 808s 367ms/step - loss: 0.1521 - accuracy: 0.9614 - val_loss: 0.4042 - val_accuracy: 0.9045
Epoch 6/10
2197/2197 [==============================] - 811s 369ms/step - loss: 0.1029 - accuracy: 0.9708 - val_loss: 0.7692 - val_accuracy: 0.8265
Epoch 7/10
2197/2197 [==============================] - 812s 369ms/step - loss: 0.0771 - accuracy: 0.9770 - val_